# PSF estimation

This notebook calculates the empirical PSF (Point Spread Function) from images in different bands. It's used as a preliminary step to SourceXtractor++, which requires PSF for source modeling. 

To calculate the PSFs, it first detects, using SExtractor, point-like sources in an image (by default the F200W band for JWST, as it gave the best results during testing), and then uses this catalog of sources to extract vignets of the point-like sources in all the bands. Then, it uses PSFEx to create the PSF for each band.

To work, this notebook requires you to have [PSFEx](https://psfex.readthedocs.io/en/latest/) and [SExtractor](https://sextractor.readthedocs.io/en/latest/Introduction.html) installed on your machine.

In [2]:
import glob
import re
import boto3
import sys
import dja_sepp

In [3]:
# Choice of the field (for file organization)
field = 'GDS'

## Point-like sources detection

This step consists in two sub-steps:
1. Running SExtractor on the image in the chosen band for point-like sources detectino (by default, F200W for JWST)
2. Detecting point-like sources in a MU_MAX v. MAG_AUTO plot. MU_MAX is the maximum pixel value for each source, given in mag.arcsec^{-2}, and MAG_AUTO is a measure of the total source magnitude, in mag. Point-like sources tend to fall on a straight line, making it easy to select them.

*N.B.: The selection of the point-like sources makes use of a linear regression to find the line, and a threshold `y_max` to exclude extended sources and allow for a good linear regression. This parameter may need to be tweaked for your images. You can use the "Star line detection after galaxy cluster removal" image to see the threshold.*

In [6]:
# F200W Point-like sources detection
filter = 'f200w'

dja_sepp.sextractor.extract_stars(detect_img      = glob.glob(f"/FlashStorage/image/{field}/*{filter}*sci*.fits")[0], \
                                  weight_img      = glob.glob(f"/FlashStorage/image/{field}/*{filter}*wht*.fits")[0], \
                                  output_cat      = f"/FlashStorage/catalog/{field}/{field}_drc_cat.fits", \
                                  output_cat_star = f"/FlashStorage/catalog/{field}/{field}_drc_cat_star.fits", \
                                  config_file     = "/home/ec2-user/DAWN/DJA-SEpp/config/PSFEx-Cat-JWST.sex", \
                                  params_file     = "/home/ec2-user/DAWN/DJA-SEpp/config/PSFEx-Cat-JWST-SW.param", \
                                  dir_chckimg     = f"/FlashStorage/catalog/{field}/checkimages", \
                                  detect_thresh = 8.0, \
                                  y_max = -5.0, \
                                  save_chckimg = False, plot = True, clean = True, verbose = True)

IndexError: list index out of range

In [ ]:
# Save star catalog to S3
dja_sepp.utils.save_S3(f"/FlashStorage/catalog/{field}/{field}_drc_cat_star.fits", 'aurelien-sepp', f"catalog/{field}")

## Calculate PSF for each band

The next step is, for each band, to : 
1. run SExtractor on the different bands,
2. cross-match this catalog with the previous point-like sources catalog,
3. run PSFEx on this cross-matched catalog to create the PSF for this band.

In [ ]:
# Filter list (for file organization and easy access)

# filter_list = ['f090w', 'f115w', 'f150w', 'f182m', 'f200w', 'f210m', 'f277w', 'f335m', 'f356w', 'f410m', 'f444w']
filter_list = [re.search('(f\d+\w+)', filename).group(1) for filename in glob.glob(f"/FlashStorage/image/{field}/*sci*")]
filter_list.sort()
filter_list

In [ ]:
# Run SExtractor, cross-match and PSFEx for each band
for filter in filter_list:
    print(filter.upper())
    print("Running SExtractor")
    dja_sepp.sextractor.extract_stars_catalog(detect_img      = glob.glob(f"/FlashStorage/image/{field}/*{filter}*sci.fits")[0], \
                                              weight_img      = glob.glob(f"/FlashStorage/image/{field}/*{filter}*wht.fits")[0], \
                                              output_cat      = f"/FlashStorage/catalog/{field}/{field}-{filter}-clear_drc_cat.fits", \
                                              output_cat_star = f"/FlashStorage/catalog/{field}/{field}-{filter}-clear_drc_cat_star.fits", \
                                              input_cat_star  = f"/FlashStorage/catalog/{field}/{field}_drc_cat_star.fits", \
                                              config_file     = "/home/ec2-user/DAWN/DJA-SEpp/config/PSFEx-Cat-JWST.sex", \
                                              params_file     = "/home/ec2-user/DAWN/DJA-SEpp/config/PSFEx-Cat-JWST-SW.param", \
                                              dir_chckimg     = f"/FlashStorage/checkimages/{field}", \
                                              detect_thresh = 2.5, \
                                              save_chckimg = False, plot = False, clean = True, run_sex = True, verbose = True)
    print("Running PSFEX")
    dja_sepp.psfex.run_psfex(cat_name    = f"/FlashStorage/catalog/{field}/{field}-{filter}-clear_drc_cat_star.fits",
                             output_cat  = f"/FlashStorage/catalog/{field}/{field}-{filter}-clear_drc_cat_star_psf.fits",
                             dir_chckimg = f"/FlashStorage/checkimages/{field}",
                             dir_psfex   = f"/FlashStorage/psfex/{field}",
                             config_file = "/home/ec2-user/DAWN/DJA-SEpp/config/psfex_default.conf",
                             fwhm_range  = '1.0, 10.0',
                             save_psf_png = True,
                             verbose = True)

## Visualize all PSFs

With the PSFs created for all the bands, it can be useful to visualize them to inspect for any defects or artefacts.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
from matplotlib import patheffects
from astropy.io import fits
from dja_sepp.utils import color_dict, channel_dict, channel_color_dict

In [ ]:
field = 'GDS'
psf_list = glob.glob(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/{field}/*star_psf.psf")
filter_list = [re.search("f{1}\d+\w{1}",psf).group() for psf in psf_list]
psf_list, filter_list = zip(*sorted(zip(psf_list, filter_list)))

In [ ]:
# Auto-sized plot to display all the PSFs
channel, count = np.unique([channel_dict[filter.upper()] for filter in filter_list], return_counts=True)
channel_count = dict(zip(channel, count))

w = min(5,channel_count[max(channel_count, key=channel_count.get)])
h = sum([-(channel_count[channel]//-w) for channel in channel_count])

fig, axs = plt.subplots(h,w,figsize=(3*w+1,3*h+1), sharex=True, sharey=True, gridspec_kw = {'wspace':0, 'hspace':0})

for i, filter in enumerate(filter_list):
    image = fits.open(psf_list[i])

    if (len(channel_count)==2) & (channel_dict[filter.upper()]=="LW"):
        i += max(0, w-(channel_count["SW"]%w))
    ax = axs.flatten()[i] if type(axs)==np.ndarray else axs

    psf = image[1].data[0][0][0]
    image.close()

    ax.set_axis_off()
    ax.imshow(psf, origin='lower', cmap='bone_r', norm=SymLogNorm(linthresh=1e-5, linscale=0.8))
    ax.text(0.06, 0.9, filter.upper(), transform=ax.transAxes, 
            fontsize='x-large', fontweight='bold', 
            color=color_dict[filter.upper()],
            path_effects=[patheffects.withStroke(linewidth=1, foreground="k")])
    ax.text(0.82, 0.89, channel_dict[filter.upper()], transform=ax.transAxes, 
            fontsize='xx-large', fontweight='bold', 
            color=channel_color_dict[channel_dict[filter.upper()]],
            path_effects=[patheffects.withStroke(linewidth=1, foreground="k")])
    
fig.tight_layout()
# fig.savefig(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/PSF_summary_{field}.pdf", bbox_inches='tight', pad_inches=0)
# fig.savefig(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/PSF_summary_{field}.png", bbox_inches='tight', pad_inches=0, dpi=100)
plt.show()

### Save all PSFs to make GIF

It is even possible to extract all the PSFs images individually, as png, to make a GIF and see how it grows with wavelength.

In [ ]:
rescale_psf = True # To rescale images between LW and SW to account for different pixel scales
for filter in filter_list:
    image = fits.open(glob.glob(f"/FlashStorage/psfex/{field}/*{filter}*star_psf.psf")[0])
    psf = image[1].data[0][0][0]
    image.close()

    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_axis_off()
    if rescale_psf:
        extent = (-100,100,-100,100) if channel_dict[filter.upper()]=='SW' else (-200,200,-200,200) # Same pixel scale
    else:
        extent = (-100,100,-100,100) # No scaling
    ax.imshow(psf, origin='lower', cmap='bone_r', norm=SymLogNorm(linthresh=1e-5, linscale=0.8), extent=extent)
    ax.set_xlim(-100,100)
    ax.set_ylim(-100,100)
    ax.text(0.04, 0.92, filter.upper(), transform=ax.transAxes, 
            fontsize='xx-large', fontweight='bold', 
            color=color_dict[filter.upper()],
            path_effects=[patheffects.withStroke(linewidth=1, foreground="k")])
    ax.text(0.84, 0.91, channel_dict[filter.upper()], transform=ax.transAxes, 
            fontsize=24, fontweight='bold', 
            color=channel_color_dict[channel_dict[filter.upper()]],
            path_effects=[patheffects.withStroke(linewidth=1, foreground="k")])
    # if rescale_psf:
    #     fig.savefig(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/scaled_{filter}.png", bbox_inches='tight', pad_inches=0, dpi=100) # Same pixel scale
    # else:
    #     fig.savefig(f"/home/ec2-user/DAWN/DJA-SEpp/psfex/GDS/{filter}.png", bbox_inches='tight', pad_inches=0, dpi=100) # No scaling
    plt.show()